In [1]:
## import relevant packages
import numpy as np
import pandas as pd
import urllib.request as urllib
import requests
from bs4 import BeautifulSoup

In [2]:
## webscraping cell

## make dataframes to be filled

draft_round = []
draft_pick = []
name = []
team = []
manager = []
position = []
player_team = []
year = []

## url setup
site1 = 'https://fantasy.nfl.com/league/2461806/history/'
site2= '/draftresults?draftResultsDetail='
site3 = '&draftResultsTab=round&draftResultsType=results'

## loop from round 1 to round 16
for z in range(2020,2021):
    for y in range(1,17):
        years = str(z)
        round_of_draft = str(y)
        
        
        
        ## create soup object for every week of every season
        site = site1 + years + site2 + round_of_draft + site3 
        html = urllib.urlopen(site)
        soup = BeautifulSoup(html.read())
        
        ## isolate html containing matchup data       
        draft_stats = soup.find("div",{"class": "wrap"})
        li_tags = draft_stats.find_all("li")
        for i in range(0,len(li_tags),2):
            year.append(int(z))
            draft_round.append(int(y))
            if li_tags[i].find("span",{"class": "count"}) is not None:
                pick_count = li_tags[i].find("span",{"class": "count"}).get_text().replace(".","")
                draft_pick.append(pick_count)
            else:
                draft_pick.append("N/A")
            
            player_name = li_tags[i].find("a",{"class": "playerName"}).get_text()
            name.append(player_name)
            
            drafted_team = li_tags[i].find("a",{"class": "teamName"}).get_text()
            team.append(drafted_team)
            
            coach = li_tags[i].find("li",{"class": "first"}).get_text()
            manager.append(coach)
            
            if li_tags[i].find("em") is not None:
                player_position = li_tags[i].find("em").get_text().split()[0]
                position.append(player_position)

                if len(li_tags[i].find("em").get_text().split()) > 1:
                    real_life_team = li_tags[i].find("em").get_text().split()[2]
                    player_team.append(real_life_team)

                else:
                    player_team.append("N/A")
            else:
                position.append("N/A")
                player_team.append("N/A")      

In [3]:
## create main dataframe
df = pd.DataFrame()
df["Draft Round"] = draft_round
df["Draft Pick"] = draft_pick
df["Draft Pick"] = df["Draft Pick"].astype(int)
df["Name"] = name
df["Team"] = team
df["Manager"] = manager
df["Position"] = position
df["Player Team"] = player_team
df["Year"] = year

In [4]:
stats2020 = pd.read_csv('2020Stats.csv', encoding='utf-8')

In [5]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
df

,Draft Round,Draft Pick,Name,Team,Manager,Position,Player Team,Year
0,1,1,Christian McCaffrey,Ishaan,Ishaan,RB,CAR,2020
1,1,2,Saquon Barkley,Number Juan,Mikey,RB,NYG,2020
2,1,3,Michael Thomas,Dreams are For Idiots,Scott,WR,NO,2020
3,1,4,Alvin Kamara,her,Krishanu,RB,NO,2020
4,1,5,Derrick Henry,Belichick Yourself,"Chris, Kanish",RB,TEN,2020
5,1,6,Ezekiel Elliott,C9H13NO3,Ayush,RB,DAL,2020
6,1,7,Dalvin Cook,The Wang Gang,Steve,RB,MIN,2020
7,1,8,Clyde Edwards-Helaire,Bok Choy Babes,Zach,RB,KC,2020
8,1,9,Davante Adams,Hernandez Hit Men,Joe,WR,GB,2020
9,1,10,DeAndre Hopkins,4-0 Dream,Ankith,WR,ARI,2020


In [6]:
## merge df with 2020 stats
df = df.merge(stats2020, how="left", on='Name')

In [7]:
## add new columns that provide new perspectives
df["PPRound"] = df["Points"] / (30 - df["Draft Round"])
df["PPPick"] = df["Points"] / (300 - df["Draft Pick"])
df["Games Missed"] = 15 - df["Games Played"]
df["Points Lost"] = df["Games Missed"] * df["PPG"] 

## QB-less dataframe
noQB = df[df["Position"] != "QB"]

In [8]:
## summary of manager's drafts
df.groupby("Manager").sum().sort_values("Points", ascending=False)

,Draft Round,Draft Pick,Year,Rank,Points,Games Played,PPG,PPRound,PPPick,Games Missed,Points Lost
Manager,,,,,,,,,,,
Steve,136,1544,32320,1845,2524.6,213,188.1,115.977443,12.433022,27,294.4
"Alex, Davis",136,1544,32320,2125,2401.9,203,186.4,112.550401,12.117023,37,392.5
Ankith,136,1544,32320,2304,2358.2,215,170.7,113.791593,12.342255,25,198.5
Ayush,136,1544,32320,2334,2352.7,199,180.6,107.680535,11.581230,41,353.4
Krishanu,136,1544,32320,2264,2315.7,201,176.5,103.278987,10.962672,39,333.7
Nandha,136,1544,32320,2510,2246.0,187,182.6,104.796829,11.183894,53,491.4
Mikey,136,1544,32320,2632,2216.0,201,166.2,101.402863,10.883852,39,277.0
"Chris, Kanish",136,1544,32320,2885,2165.2,202,161.4,100.687657,10.804723,38,255.7
Joe,136,1544,32320,2781,2084.0,196,161.0,91.486248,9.686928,44,327.8


In [9]:
## summary of manager's drafts
df.groupby("Manager").sum().sort_values("PPPick", ascending=False)

,Draft Round,Draft Pick,Year,Rank,Points,Games Played,PPG,PPRound,PPPick,Games Missed,Points Lost
Manager,,,,,,,,,,,
Steve,136,1544,32320,1845,2524.6,213,188.1,115.977443,12.433022,27,294.4
Ankith,136,1544,32320,2304,2358.2,215,170.7,113.791593,12.342255,25,198.5
"Alex, Davis",136,1544,32320,2125,2401.9,203,186.4,112.550401,12.117023,37,392.5
Ayush,136,1544,32320,2334,2352.7,199,180.6,107.680535,11.581230,41,353.4
Scott,136,1544,32320,2784,2055.9,200,158.6,101.954539,11.202179,40,324.1
Nandha,136,1544,32320,2510,2246.0,187,182.6,104.796829,11.183894,53,491.4
Krishanu,136,1544,32320,2264,2315.7,201,176.5,103.278987,10.962672,39,333.7
Mikey,136,1544,32320,2632,2216.0,201,166.2,101.402863,10.883852,39,277.0
"Chris, Kanish",136,1544,32320,2885,2165.2,202,161.4,100.687657,10.804723,38,255.7


In [10]:
## highest scoring players drafted in double digit rounds
df[df['Draft Round'] > 9].sort_values("Points", ascending = False)

,Draft Round,Draft Pick,Name,Team,Manager,Position,Player Team,Year,Rank,TM,PSN,Points,Games Played,PPG,PPRound,PPPick,Games Missed,Points Lost
148,13,149,Ryan Tannehill,Belichick Yourself,"Chris, Kanish",QB,TEN,2020,8,TEN,QB,322.4,15,21.5,18.964706,2.135099,0,0.0
170,15,171,Kirk Cousins,Dreams are For Idiots,Scott,QB,MIN,2020,16,MIN,QB,284.9,15,19.0,18.993333,2.208527,0,0.0
134,12,135,Ben Roethlisberger,4-0 Dream,Ankith,QB,PIT,2020,17,PIT,QB,277.2,15,18.5,15.400000,1.680000,0,0.0
182,16,183,Jared Goff,4-0 Dream,Ankith,QB,LA,2020,22,DET,QB,253.0,15,16.9,18.071429,2.162393,0,0.0
108,10,109,Matthew Stafford,Showtyme,Nandha,QB,DET,2020,23,LAR,QB,246.9,15,16.5,12.345000,1.292670,0,0.0
167,14,168,Baker Mayfield,Ishaan,Ishaan,QB,CLE,2020,25,CLE,QB,240.1,15,16.0,15.006250,1.818939,0,0.0
173,15,174,Cam Newton,C9H13NO3,Ayush,QB,NE,2020,27,NE,QB,232.3,14,16.6,15.486667,1.843651,1,16.6
126,11,127,Justin Jefferson,The Wang Gang,Steve,WR,MIN,2020,35,MIN,WR,212.4,15,14.2,11.178947,1.227746,0,0.0
155,13,156,Joe Burrow,Showtyme,Nandha,QB,CIN,2020,51,CIN,QB,178.6,10,17.9,10.505882,1.240278,5,89.5
109,10,110,Antonio Gibson,Aint Talkin' Bout Chubb,"Alex, Davis",RB,WAS,2020,53,WAS,RB,175.6,13,13.5,8.780000,0.924211,2,27.0


In [11]:
## highest scoring non-QBs drafted in double digit rounds
noQB[noQB['Draft Round'] > 9].sort_values("Points", ascending = False)

,Draft Round,Draft Pick,Name,Team,Manager,Position,Player Team,Year,Rank,TM,PSN,Points,Games Played,PPG,PPRound,PPPick,Games Missed,Points Lost
126,11,127,Justin Jefferson,The Wang Gang,Steve,WR,MIN,2020,35,MIN,WR,212.4,15,14.2,11.178947,1.227746,0,0.0
109,10,110,Antonio Gibson,Aint Talkin' Bout Chubb,"Alex, Davis",RB,WAS,2020,53,WAS,RB,175.6,13,13.5,8.780000,0.924211,2,27.0
127,11,128,CeeDee Lamb,Bok Choy Babes,Zach,WR,DAL,2020,55,DAL,WR,173.6,15,11.6,9.136842,1.009302,0,0.0
166,14,167,Robby Anderson,Number Juan,Mikey,WR,CAR,2020,58,CAR,WR,171.2,15,11.4,10.700000,1.287218,0,0.0
163,14,164,Younghoe Koo,Belichick Yourself,"Chris, Kanish",K,ATL,2020,73,ATL,K,159.0,14,11.4,9.937500,1.169118,1,11.4
168,15,169,Curtis Samuel,Ishaan,Ishaan,WR,CAR,2020,75,WAS,WR,158.0,14,11.3,10.533333,1.206107,1,11.3
130,11,131,Brandon Aiyuk,Aint Talkin' Bout Chubb,"Alex, Davis",WR,SF,2020,81,SF,WR,154.5,12,12.9,8.131579,0.914201,3,38.7
146,13,147,Nyheim Hines,Dreams are For Idiots,Scott,RB,IND,2020,84,IND,RB,152.0,15,10.1,8.941176,0.993464,0,0.0
118,10,119,Pittsburgh Steelers,Number Juan,Mikey,DEF,N/A,2020,91,PIT,DST,144.0,15,9.6,7.200000,0.795580,0,0.0
140,12,141,Chase Edmonds,her,Krishanu,RB,ARI,2020,100,ARI,RB,138.0,15,9.2,7.666667,0.867925,0,0.0


In [12]:
## managers who lost most points due to injury to drafted players
df.groupby("Manager").sum().sort_values("Points Lost", ascending=False)

,Draft Round,Draft Pick,Year,Rank,Points,Games Played,PPG,PPRound,PPPick,Games Missed,Points Lost
Manager,,,,,,,,,,,
Ishaan,136,1544,32320,3105,1736.6,175,182.1,87.643910,9.657512,65,994.5
Zach,136,1544,32320,2368,1932.5,197,163.0,94.219624,10.286920,43,509.3
Nandha,136,1544,32320,2510,2246.0,187,182.6,104.796829,11.183894,53,491.4
"Alex, Davis",136,1544,32320,2125,2401.9,203,186.4,112.550401,12.117023,37,392.5
Ayush,136,1544,32320,2334,2352.7,199,180.6,107.680535,11.581230,41,353.4
Krishanu,136,1544,32320,2264,2315.7,201,176.5,103.278987,10.962672,39,333.7
Joe,136,1544,32320,2781,2084.0,196,161.0,91.486248,9.686928,44,327.8
Scott,136,1544,32320,2784,2055.9,200,158.6,101.954539,11.202179,40,324.1
Steve,136,1544,32320,1845,2524.6,213,188.1,115.977443,12.433022,27,294.4


In [13]:
## creating dataframes to showcase the highest scoring pick in each round

bests = pd.DataFrame()
for i in range(1,17):
    df1 = pd.DataFrame() 
    df1 = df[df["Points"] == df.groupby("Draft Round").max()["Points"][i]]
    bests = bests.append(df1)
    
qbless = pd.DataFrame()    
for i in range(1,17):
    df1 = pd.DataFrame() 
    df1 = noQB[noQB["Points"] == noQB.groupby("Draft Round").max()["Points"][i]]
    qbless = qbless.append(df1)
    

In [14]:
##bests
qbless

,Draft Round,Draft Pick,Name,Team,Manager,Position,Player Team,Year,Rank,TM,PSN,Points,Games Played,PPG,PPRound,PPPick,Games Missed,Points Lost
3,1,4,Alvin Kamara,her,Krishanu,RB,NO,2020,7,NO,RB,336.3,15,22.4,11.596552,1.136149,0,0.0
15,2,16,Tyreek Hill,Hernandez Hit Men,Joe,WR,KC,2020,15,KC,WR,285.4,15,19.0,10.192857,1.004930,0,0.0
29,3,30,Adam Thielen,C9H13NO3,Ayush,WR,MIN,2020,36,MIN,WR,209.3,14,15.0,7.751852,0.775185,1,15.0
37,4,38,Calvin Ridley,Aint Talkin' Bout Chubb,"Alex, Davis",WR,ATL,2020,28,ATL,WR,229.9,14,16.4,8.842308,0.877481,1,16.4
53,5,54,Stefon Diggs,C9H13NO3,Ayush,WR,BUF,2020,21,BUF,WR,254.0,15,16.9,10.160000,1.032520,0,0.0
61,6,62,David Montgomery,Aint Talkin' Bout Chubb,"Alex, Davis",RB,CHI,2020,33,CHI,RB,214.1,14,15.3,8.920833,0.899580,1,15.3
75,7,76,Terry McLaurin,her,Krishanu,WR,WAS,2020,60,WAS,WR,166.8,14,11.9,7.252174,0.744643,1,11.9
90,8,91,Brandin Cooks,C9H13NO3,Ayush,WR,HOU,2020,77,HOU,WR,157.4,14,11.2,7.154545,0.753110,1,11.2
107,9,108,Diontae Johnson,Showtyme,Nandha,WR,PIT,2020,61,PIT,WR,166.7,14,11.9,7.938095,0.868229,1,11.9
109,10,110,Antonio Gibson,Aint Talkin' Bout Chubb,"Alex, Davis",RB,WAS,2020,53,WAS,RB,175.6,13,13.5,8.780000,0.924211,2,27.0


In [15]:
## most non-QB draft picks that scored the most in their draft round

qbless.groupby("Manager").count().sort_values("Year", ascending=False)

,Draft Round,Draft Pick,Name,Team,Position,Player Team,Year,Rank,TM,PSN,Points,Games Played,PPG,PPRound,PPPick,Games Missed,Points Lost
Manager,,,,,,,,,,,,,,,,,
"Alex, Davis",4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
Ayush,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
Krishanu,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
Ishaan,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Joe,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Mikey,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Nandha,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Scott,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Steve,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [16]:
## best value picks (QBs)
df.sort_values("PPPick", ascending=False)

,Draft Round,Draft Pick,Name,Team,Manager,Position,Player Team,Year,Rank,TM,PSN,Points,Games Played,PPG,PPRound,PPPick,Games Missed,Points Lost
170,15,171,Kirk Cousins,Dreams are For Idiots,Scott,QB,MIN,2020,16,MIN,QB,284.9,15,19.0,18.993333,2.208527,0,0.0
182,16,183,Jared Goff,4-0 Dream,Ankith,QB,LA,2020,22,DET,QB,253.0,15,16.9,18.071429,2.162393,0,0.0
148,13,149,Ryan Tannehill,Belichick Yourself,"Chris, Kanish",QB,TEN,2020,8,TEN,QB,322.4,15,21.5,18.964706,2.135099,0,0.0
106,9,107,Aaron Rodgers,Aint Talkin' Bout Chubb,"Alex, Davis",QB,GB,2020,4,GB,QB,361.6,15,24.1,17.219048,1.873575,0,0.0
173,15,174,Cam Newton,C9H13NO3,Ayush,QB,NE,2020,27,NE,QB,232.3,14,16.6,15.486667,1.843651,1,16.6
167,14,168,Baker Mayfield,Ishaan,Ishaan,QB,CLE,2020,25,CLE,QB,240.1,15,16.0,15.006250,1.818939,0,0.0
84,8,85,Josh Allen,Showtyme,Nandha,QB,BUF,2020,2,BUF,QB,385.4,15,25.7,17.518182,1.792558,0,0.0
134,12,135,Ben Roethlisberger,4-0 Dream,Ankith,QB,PIT,2020,17,PIT,QB,277.2,15,18.5,15.400000,1.680000,0,0.0
54,5,55,Kyler Murray,The Wang Gang,Steve,QB,ARI,2020,1,ARI,QB,386.9,15,25.8,15.476000,1.579184,0,0.0
70,6,71,Deshaun Watson,Number Juan,Mikey,QB,HOU,2020,6,HOU,QB,349.6,15,23.3,14.566667,1.526638,0,0.0


In [17]:
## best value picks (noQBs)
noQB.sort_values("PPPick", ascending=False)

,Draft Round,Draft Pick,Name,Team,Manager,Position,Player Team,Year,Rank,TM,PSN,Points,Games Played,PPG,PPRound,PPPick,Games Missed,Points Lost
166,14,167,Robby Anderson,Number Juan,Mikey,WR,CAR,2020,58,CAR,WR,171.2,15,11.4,10.700000,1.287218,0,0.0
126,11,127,Justin Jefferson,The Wang Gang,Steve,WR,MIN,2020,35,MIN,WR,212.4,15,14.2,11.178947,1.227746,0,0.0
168,15,169,Curtis Samuel,Ishaan,Ishaan,WR,CAR,2020,75,WAS,WR,158.0,14,11.3,10.533333,1.206107,1,11.3
163,14,164,Younghoe Koo,Belichick Yourself,"Chris, Kanish",K,ATL,2020,73,ATL,K,159.0,14,11.4,9.937500,1.169118,1,11.4
3,1,4,Alvin Kamara,her,Krishanu,RB,NO,2020,7,NO,RB,336.3,15,22.4,11.596552,1.136149,0,0.0
174,15,175,Indianapolis Colts,The Wang Gang,Steve,DEF,N/A,2020,103,IND,DST,137.0,15,9.1,9.133333,1.096000,0,0.0
6,1,7,Dalvin Cook,The Wang Gang,Steve,RB,MIN,2020,11,MIN,RB,315.8,14,22.6,10.889655,1.077816,1,22.6
53,5,54,Stefon Diggs,C9H13NO3,Ayush,WR,BUF,2020,21,BUF,WR,254.0,15,16.9,10.160000,1.032520,0,0.0
127,11,128,CeeDee Lamb,Bok Choy Babes,Zach,WR,DAL,2020,55,DAL,WR,173.6,15,11.6,9.136842,1.009302,0,0.0
15,2,16,Tyreek Hill,Hernandez Hit Men,Joe,WR,KC,2020,15,KC,WR,285.4,15,19.0,10.192857,1.004930,0,0.0


In [18]:
## biggest busts from first two rounds
df[((df["Draft Round"] == 1) | (df["Draft Round"] == 2)) & (df["Games Played"] > 9)].sort_values("PPG", ascending=True)

,Draft Round,Draft Pick,Name,Team,Manager,Position,Player Team,Year,Rank,TM,PSN,Points,Games Played,PPG,PPRound,PPPick,Games Missed,Points Lost
19,2,20,Chris Godwin,Belichick Yourself,"Chris, Kanish",WR,TB,2020,116,TB,WR,130.7,11,11.9,4.667857,0.466786,4,47.6
7,1,8,Clyde Edwards-Helaire,Bok Choy Babes,Zach,RB,KC,2020,76,KC,RB,158.0,13,12.2,5.448276,0.541096,2,24.4
12,2,13,Kenyan Drake,Showtyme,Nandha,RB,ARI,2020,56,LV,RB,173.1,14,12.4,6.182143,0.603136,1,12.4
20,2,21,Miles Sanders,her,Krishanu,RB,PHI,2020,78,PHI,RB,156.4,12,13.0,5.585714,0.560573,3,39.0
5,1,6,Ezekiel Elliott,C9H13NO3,Ayush,RB,DAL,2020,48,DAL,RB,184.1,14,13.2,6.348276,0.626190,1,13.2
13,2,14,Josh Jacobs,Aint Talkin' Bout Chubb,"Alex, Davis",RB,LV,2020,45,LV,RB,193.9,14,13.9,6.925000,0.677972,1,13.9
9,1,10,DeAndre Hopkins,4-0 Dream,Ankith,WR,ARI,2020,31,ARI,WR,224.8,15,15.0,7.751724,0.775172,0,0.0
10,1,11,Nick Chubb,Aint Talkin' Bout Chubb,"Alex, Davis",RB,CLE,2020,50,CLE,RB,182.9,11,16.6,6.306897,0.632872,4,66.4
14,2,15,Aaron Jones,4-0 Dream,Ankith,RB,GB,2020,32,GB,RB,218.9,13,16.8,7.817857,0.768070,2,33.6
22,2,23,Travis Kelce,Number Juan,Mikey,TE,KC,2020,19,KC,TE,260.3,15,17.4,9.296429,0.939711,0,0.0


In [19]:
## worst value picks in the first hundred picks
df[(df["Draft Pick"] < 101) & (df["Games Played"] > 9)].sort_values("PPPick", ascending=True)

,Draft Round,Draft Pick,Name,Team,Manager,Position,Player Team,Year,Rank,TM,PSN,Points,Games Played,PPG,PPRound,PPPick,Games Missed,Points Lost
43,4,44,Mark Ingram,Belichick Yourself,"Chris, Kanish",RB,BAL,2020,327,HOU,RB,46.0,10,4.6,1.769231,0.179688,5,23.0
99,9,100,Matt Breida,her,Krishanu,RB,MIA,2020,354,BUF,RB,37.5,11,3.4,1.785714,0.187500,4,13.6
59,5,60,Zach Ertz,Showtyme,Nandha,TE,PHI,2020,292,PHI,TE,56.4,10,5.6,2.256000,0.235000,5,28.0
50,5,51,Kerryon Johnson,Dreams are For Idiots,Scott,RB,DET,2020,282,PHI,RB,62.3,14,4.5,2.492000,0.250201,1,4.5
33,3,34,Le'Veon Bell,4-0 Dream,Ankith,RB,KC,2020,265,FA,RB,66.6,11,6.1,2.466667,0.250376,4,24.4
76,7,77,Phillip Lindsay,Belichick Yourself,"Chris, Kanish",RB,DEN,2020,281,HOU,RB,62.5,11,5.7,2.717391,0.280269,4,22.8
60,6,61,Cam Akers,Showtyme,Nandha,RB,LA,2020,218,LAR,RB,87.7,11,8.0,3.654167,0.366946,4,32.0
86,8,87,James White,4-0 Dream,Ankith,RB,NE,2020,229,NE,RB,81.0,13,6.2,3.681818,0.380282,2,12.4
85,8,86,Anthony Miller,Aint Talkin' Bout Chubb,"Alex, Davis",WR,CHI,2020,226,HOU,WR,81.9,15,5.5,3.722727,0.382710,0,0.0
81,7,82,A.J. Green,4-0 Dream,Ankith,WR,CIN,2020,217,ARI,WR,87.8,15,5.9,3.817391,0.402752,0,0.0


In [20]:
##df.groupby("Position").count().sort_values("Year", ascending=False)
## seeing if any manager has 4+ players from one team
df[df["Manager"] == "Ankith"].groupby("Player Team").count().sort_values("Year", ascending=False)

,Draft Round,Draft Pick,Name,Team,Manager,Position,Year,Rank,TM,PSN,Points,Games Played,PPG,PPRound,PPPick,Games Missed,Points Lost
Player Team,,,,,,,,,,,,,,,,,
NE,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
GB,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
LA,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
TB,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
ARI,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
CIN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
DET,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
IND,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
KC,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [21]:
## see best drafted starting lineups

best_lineups = pd.DataFrame()
manager = []
total = []
QB = []
RB1 = []
RB2 = []
WR1 = []
WR2 = []
RB3 = []
WR3 = []
FLEX = []
TE = []

managers = ['Alex, Davis', 'Steve', 'Ayush','Nandha', 'Krishanu','Chris, Kanish','Mikey','Joe','Ishaan ','Zach','Scott','Ankith']

for x in managers:
    manager.append(x)
    QB.append(df[(df["Manager"] == x) & (df["Position"] == "QB")].max()["Points"])
    RB1.append(df[(df["Manager"] == x) & (df["Position"] == "RB")].max()["Points"])
    RB2.append(df[(df["Manager"] == x) & (df["Position"] == "RB")]["Points"].nlargest(2).min())
    WR1.append(df[(df["Manager"] == x) & (df["Position"] == "WR")].max()["Points"])
    WR2.append(df[(df["Manager"] == x) & (df["Position"] == "WR")]["Points"].nlargest(2).min())
    RB3.append(df[(df["Manager"] == x) & (df["Position"] == "RB")]["Points"].nlargest(3).min())
    WR3.append(df[(df["Manager"] == x) & (df["Position"] == "WR")]["Points"].nlargest(3).min())
    TE.append(df[(df["Manager"] == x) & (df["Position"] == "TE")].max()["Points"])

best_lineups["Manager"] = manager
best_lineups["QB"] = QB
best_lineups["RB1"] = RB1
best_lineups["RB2"] = RB2
best_lineups["WR1"] = WR1
best_lineups["WR2"] = WR2
best_lineups["RB3"] = RB3 
best_lineups["WR3"] = WR3
best_lineups["FLEX"] = np.where(best_lineups["RB3"] > best_lineups["WR3"], best_lineups["RB3"], best_lineups["WR3"])
best_lineups["TE"] = TE 
best_lineups["total"] = best_lineups["QB"] + best_lineups["RB1"] + best_lineups["RB2"] + best_lineups["WR1"] + best_lineups["WR2"] + best_lineups["FLEX"] + best_lineups["TE"]


In [22]:
## best season best ball team
best_lineups.sort_values("total", ascending=False)

,Manager,QB,RB1,RB2,WR1,WR2,RB3,WR3,FLEX,TE,total
4,Krishanu,270.6,336.3,158.5,186.4,183.7,156.4,166.8,166.8,202.9,1505.2
2,Ayush,380.3,194.9,184.1,254.0,209.3,138.4,157.4,157.4,110.3,1490.3
1,Steve,386.9,315.8,143.5,212.4,165.9,124.2,151.4,151.4,113.8,1489.7
0,"Alex, Davis",361.6,214.1,193.9,229.9,162.4,182.9,154.5,182.9,136.4,1481.2
7,Joe,213.4,163.4,124.7,287.3,285.4,110.3,226.2,226.2,93.4,1393.8
6,Mikey,349.6,122.8,28.4,207.5,188.4,12.4,171.2,171.2,260.3,1328.2
3,Nandha,385.4,173.1,89.1,207.2,170.1,88.8,166.7,166.7,125.6,1317.2
11,Ankith,318.7,218.9,152.3,224.8,162.7,81.0,87.8,87.8,122.9,1288.1
5,"Chris, Kanish",354.5,288.6,74.2,194.5,130.7,62.5,107.1,107.1,106.8,1256.4
10,Scott,316.5,152.0,146.7,195.1,137.4,108.6,112.8,112.8,115.3,1175.8


In [26]:
df[df["Manager"] == "Ayush"]

,Draft Round,Draft Pick,Name,Team,Manager,Position,Player Team,Year,Rank,TM,PSN,Points,Games Played,PPG,PPRound,PPPick,Games Missed,Points Lost
5,1,6,Ezekiel Elliott,C9H13NO3,Ayush,RB,DAL,2020,48,DAL,RB,184.1,14,13.2,6.348276,0.626190,1,13.2
18,2,19,Patrick Mahomes,C9H13NO3,Ayush,QB,KC,2020,3,KC,QB,380.3,15,25.4,13.582143,1.353381,0,0.0
29,3,30,Adam Thielen,C9H13NO3,Ayush,WR,MIN,2020,36,MIN,WR,209.3,14,15.0,7.751852,0.775185,1,15.0
42,4,43,James Conner,C9H13NO3,Ayush,RB,PIT,2020,98,ARI,RB,138.4,12,11.5,5.323077,0.538521,3,34.5
53,5,54,Stefon Diggs,C9H13NO3,Ayush,WR,BUF,2020,21,BUF,WR,254.0,15,16.9,10.160000,1.032520,0,0.0
66,6,67,Kareem Hunt,C9H13NO3,Ayush,RB,CLE,2020,43,CLE,RB,194.9,15,13.0,8.120833,0.836481,0,0.0
77,7,78,Julian Edelman,C9H13NO3,Ayush,WR,NE,2020,328,FA,WR,45.7,6,7.6,1.986957,0.205856,9,68.4
90,8,91,Brandin Cooks,C9H13NO3,Ayush,WR,HOU,2020,77,HOU,WR,157.4,14,11.2,7.154545,0.753110,1,11.2
101,9,102,Hayden Hurst,C9H13NO3,Ayush,TE,ATL,2020,161,ATL,TE,110.3,15,7.4,5.252381,0.557071,0,0.0
114,10,115,Henry Ruggs III,C9H13NO3,Ayush,WR,LV,2020,262,LV,WR,67.8,12,5.7,3.390000,0.366486,3,17.1
